In [44]:
import xarray as xr
import pandas as pd
import netCDF4 as nc
import tqdm
import csv



In [45]:
filePath = '../Dataset/climate_dataset/TempAir.nc'  
dSet = nc.Dataset(filePath)  

In [46]:
for dimension in dSet.dimensions.values():  
    print(dimension)  

<class 'netCDF4._netCDF4.Dimension'>: name = 'time', size = 744
<class 'netCDF4._netCDF4.Dimension'>: name = 'lon', size = 720
<class 'netCDF4._netCDF4.Dimension'>: name = 'lat', size = 360


In [47]:
import netCDF4 as nc
import csv
import tqdm
import xarray as xr

ds = xr.open_dataset("../Dataset/climate_dataset/TempAir.nc")
nc_file = ds

csv_file = open('output.csv', 'w', newline='')
csv_writer = csv.writer(csv_file)

time = nc_file.variables['time']
lon = nc_file.variables['lon']
lat = nc_file.variables['lat']
csv_writer.writerow(['Time', 'Lon', 'Lat', 'Value'])

data_variable = nc_file.variables['Tair']

# Reduce the size of the 'time' dimension to 5
time_indices = range(0, len(time), len(time)//5)

for t in tqdm.tqdm(time_indices):
    for lat_index in range(len(lat)):
        for lon_index in range(len(lon)):
            time_value = time[t]
            lat_value = lat[lat_index]
            lon_value = lon[lon_index]
            data_value = data_variable[t, lat_index, lon_index]
            csv_writer.writerow([time_value.values, lon_value.values, lat_value.values, data_value.values])


csv_file.close()
nc_file.close()



  0%|          | 0/6 [00:00<?, ?it/s]

100%|██████████| 6/6 [05:51<00:00, 58.63s/it]

Total Dataset Size: 192844800


In [48]:
df=pd.read_csv('output.csv')

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1555200 entries, 0 to 1555199
Data columns (total 4 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   Time    1555200 non-null  object 
 1   Lon     1555200 non-null  float64
 2   Lat     1555200 non-null  float64
 3   Value   557334 non-null   float64
dtypes: float64(3), object(1)
memory usage: 47.5+ MB


In [51]:
train=df.drop(columns=['Time'])

In [52]:
train

,Lon,Lat,Value
0,-179.75,-89.75,243.30138
1,-179.25,-89.75,243.30110
2,-178.75,-89.75,243.31395
3,-178.25,-89.75,243.31410
4,-177.75,-89.75,243.32626
...,...,...,...
1555195,177.75,89.75,NaN
1555196,178.25,89.75,NaN
1555197,178.75,89.75,NaN
1555198,179.25,89.75,NaN


In [53]:
df['Time']=pd.to_datetime(df['Time'])


In [54]:
#!pip install sklearn==1.2.1

In [55]:

import sklearn.neighbors._base
import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base

In [56]:
from missingpy import MissForest
imputer = MissForest(n_estimators=100, max_iter=10, n_jobs=-1)
X_imputed = imputer.fit_transform(train)

/Users/farahmurtaza/Library/Python/3.9/lib/python/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/farahmurtaza/Library/Python/3.9/lib/python/site-packages/sklearn/utils/validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/farahmurtaza/Library/Python/3.9/lib/python/site-packages/sklearn/utils/validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/farahmurtaza/Library/Python/3.9/lib/python/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be

Iteration: 0


/Users/farahmurtaza/Library/Python/3.9/lib/python/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Users/farahmurtaza/Library/Python/3.9/lib/python/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


Iteration: 1


/Users/farahmurtaza/Library/Python/3.9/lib/python/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Users/farahmurtaza/Library/Python/3.9/lib/python/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


Iteration: 2


In [57]:
from sklearn.preprocessing import StandardScaler

In [58]:
scaler = StandardScaler()

scaled_data = scaler.fit_transform(X_imputed)

In [59]:
df_final_data=pd.DataFrame(scaled_data,columns=['Lon','Lat','Value'])

In [60]:
df_final_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1555200 entries, 0 to 1555199
Data columns (total 3 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   Lon     1555200 non-null  float64
 1   Lat     1555200 non-null  float64
 2   Value   1555200 non-null  float64
dtypes: float64(3)
memory usage: 35.6 MB


In [61]:
df_final_data['Time']=df['Time']

In [62]:
df_final_data

,Lon,Lat,Value,Time
0,-1.729647,-1.727246,-1.558152,2019-01-01 00:00:00
1,-1.724836,-1.727246,-1.558166,2019-01-01 00:00:00
2,-1.720024,-1.727246,-1.557558,2019-01-01 00:00:00
3,-1.715213,-1.727246,-1.557551,2019-01-01 00:00:00
4,-1.710402,-1.727246,-1.556976,2019-01-01 00:00:00
...,...,...,...,...
1555195,1.710402,1.727246,-1.496074,2019-01-31 20:00:00
1555196,1.715213,1.727246,-1.504687,2019-01-31 20:00:00
1555197,1.720024,1.727246,-1.520775,2019-01-31 20:00:00
1555198,1.724836,1.727246,-1.514635,2019-01-31 20:00:00


In [63]:
from sklearn.model_selection import train_test_split

In [64]:
train_data, test_data = train_test_split(df_final_data, test_size=0.2, random_state=42)

train_file_path = "../Dataset/climate_dataset/train_data.csv"
test_file_path = "../Dataset/climate_dataset/test_data.csv"

In [65]:
# Convert the DataFrames to CSV files
train_data.to_csv(train_file_path, index=False)
test_data.to_csv(test_file_path, index=False)